In [2]:
import geopandas as gpd

path= "/workspaces/Satelite/data/grid/CH_grid.gpkg"
grid = gpd.read_file(path)
border = "/workspaces/Satelite/data/borders.geojson"
border = gpd.read_file(border)
simplified = "/workspaces/Satelite/data/CH_simplified.gpkg"
simplified = gpd.read_file(simplified)

In [3]:
# Ensure all data has the same CRS
if grid.crs != border.crs:
    border = border.to_crs(grid.crs)
if simplified.crs != grid.crs:
    simplified = simplified.to_crs(grid.crs)

In [12]:
import geopandas as gpd
import numpy as np
from shapely.geometry import box

grid['cell_area'] = grid['geometry'].area

# Perform spatial join between grid and border to find cells intersecting the border
print('Performing spatial join to find intersecting cells...')
joined = gpd.sjoin(grid, border, how='inner', op='intersects')
print(f'Number of intersecting cells: {len(joined)}')

# Filter out cells that are not fully within the border
print('Filtering cells that are fully within the border...')
fully_within_cells = joined[joined.apply(lambda row: border.contains(row.geometry).all(), axis=1)]
print(f'Number of fully within cells: {len(fully_within_cells)}')

# Reset index to avoid conflicts in spatial join
fully_within_cells = fully_within_cells.reset_index(drop=True)
simplified = simplified.reset_index(drop=True)

# Drop index_left and index_right columns if they exist
if 'index_left' in fully_within_cells.columns:
    fully_within_cells = fully_within_cells.drop(columns=['index_left'])
if 'index_right' in fully_within_cells.columns:
    fully_within_cells = fully_within_cells.drop(columns=['index_right'])
if 'index_left' in simplified.columns:
    simplified = simplified.drop(columns=['index_left'])
if 'index_right' in simplified.columns:
    simplified = simplified.drop(columns=['index_right'])

# Perform spatial join between fully within cells and simplified parcel data
print('Performing spatial join with simplified data...')
joined_simplified = gpd.sjoin(fully_within_cells, simplified, how='inner', predicate='intersects')
print(f'Number of joined cells with simplified data: {len(joined_simplified)}')

# Ensure 'area' from the simplified data is used for calculating coverage ratio
print('Grouping joined cells by cell_id...')
grouped = joined_simplified.groupby('cell_id').agg({
    'geometry': 'first',
    'area': 'sum',
    'cell_area': 'first'
}).reset_index()

# Calculate the coverage ratio
grouped['coverage_ratio'] = grouped['area'] / grouped['cell_area']

# Filter essential cells based on coverage ratio
non_essential_cells_threshold = 0.1  # Adjust based on your requirement
essential_cells = grouped[grouped['coverage_ratio'] >= non_essential_cells_threshold]
print(f'Number of essential cells: {len(essential_cells)}')

# If there are essential cells, create the GeoDataFrame and save to file
if not essential_cells.empty:
    essential_cells['cell_id'] = np.arange(1, len(essential_cells) + 1)
    essential_cells = gpd.GeoDataFrame(essential_cells, geometry='geometry', crs=grid.crs)
    essential_cells.to_file("/workspaces/Satelite/data/grid/CH_essential_grid.gpkg", driver="GPKG")
    print('Saved essential grid cells.')
else:
    print('No essential cells to save.')

Performing spatial join to find intersecting cells...


/home/tfuser/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Number of intersecting cells: 6435
Filtering cells that are fully within the border...
Number of fully within cells: 5385
Performing spatial join with simplified data...
Number of joined cells with simplified data: 1904573
Grouping joined cells by cell_id...
Number of essential cells: 4157
Number of essential cells after size filtering: 4157
Saved essential grid cells.
